In [1]:
#import the dependinces
import pandas as pd
import csv
import numpy as np
import gmaps
import datetime
import time
from scipy import stats
from matplotlib import pyplot as plt
import os
import warnings

# Ignore unnecessary gmaps warning
warnings.filterwarnings('ignore')

In [11]:
#Import the csvs
lapd = pd.read_csv("../crime_data/Crime_Data_from_2010_to_2019.csv")
og_file = pd.read_csv("../crime_data/NYPD_Arrests_Data__Historic_.csv")

In [4]:
#Drop unessacary rows for LA
lapd.drop(columns=['Status Desc','DR_NO', 'Date Rptd','AREA ','Rpt Dist No', 'Part 1-2', 'Crm Cd','Mocodes','Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd','Weapon Used Cd','Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4','LOCATION', 'Cross Street'],axis=1,inplace=True)
lapd.columns

Index(['DATE OCC', 'TIME OCC', 'AREA NAME', 'Crm Cd Desc', 'Premis Desc',
       'Weapon Desc', 'LAT', 'LON'],
      dtype='object')

In [5]:
#Rename each crime committed in LA data
lapd['Crm Cd Desc'] = lapd['Crm Cd Desc'].replace({'VIOLATION OF COURT ORDER':'VIOLATION OF',
'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)':'VANDALISM', 'RAPE, ATTEMPTED':'RAPE',
'SHOPLIFTING - PETTY THEFT ($950 & UNDER)':'SHOPLIFTING','BURGLARY FROM VEHICLE':'BURGLARY',
'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT':'ASSAULT','THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD':'THEFT',
'BATTERY - SIMPLE ASSAULT':'BATTERY','CHILD NEGLECT (SEE 300 W.I.C.)':'CHILD','INTIMATE PARTNER - AGGRAVATED ASSAULT':'ASSAULT',
'INTIMATE PARTNER - SIMPLE ASSAULT':'ASSAULT','THEFT PLAIN - PETTY ($950 & UNDER)':'THEFT', 'ATTEMPTED ROBBERY':'ROBBERY',
'VANDALISM - MISDEAMEANOR ($399 OR UNDER)':'VANDALISM','RAPE, FORCIBLE':'RAPE','THROWING OBJECT AT MOVING VEHICLE':'VEHICLE',
'SHOPLIFTING-GRAND THEFT ($950.01 & OVER)':'SHOPLIFTING','CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT':'CHILD',
'SHOTS FIRED AT INHABITED DWELLING':'SHOTS FIRED','THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER)':'THEFT',
'VEHICLE - STOLEN':'VEHICLE','KIDNAPPING - GRAND ATTEMPT':'KIDNAPPING','THEFT PLAIN - ATTEMPT':'THEFT','THEFT, PERSON':'THEFT',
'VEHICLE - ATTEMPT STOLEN':'VEHICLE','BATTERY WITH SEXUAL CONTACT':'BATTERY','BATTERY POLICE (SIMPLE)':'BATTERY',
'CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT':'CHILD','OTHER ASSAULT':'ASSAULT', 'VIOLATION OF RESTRAINING ORDER':'VIOLATION OF',
'CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 YRS OLDER)':'CHILD','THEFT OF IDENTITY':'THEFT','ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER':'ASSAULT',
'CHILD ANNOYING (17YRS & UNDER)':'CHILD','SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH':'RAPE',
'THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)':'THEFT','LETTERS, LEWD  -  TELEPHONE CALLS, LEWD':'LEWD CONDUCT',
'INDECENT EXPOSURE':'LEWD CONDUCT','BURGLARY FROM VEHICLE, ATTEMPTED':'BURGLARY','BURGLARY, ATTEMPTED':'BURGLARY',
'THREATENING PHONE CALLS/LETTERS':'CRIMINAL THREATS','SEX,UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ':'RAPE',                                                  
'BUNCO, PETTY THEFT':'BUNCO','VIOLATION OF TEMPORARY RESTRAINING ORDER':'VIOLATION OF','BUNCO, GRAND THEFT':'BUNCO',                                                   
'EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)':'EMBEZZLEMENT','SEXUAL PENETRATION W/FOREIGN OBJECT':'RAPE','EMBEZZLEMENT, PETTY THEFT ($950 & UNDER)':'EMBEZZLEMENT',                                                   
'DEFRAUDING INNKEEPER/THEFT OF SERVICES, $400 & UNDER':'THEFT','CREDIT CARDS, FRAUD USE ($950.01 & OVER)':'FRAUD','DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $400':'THEFT',                                                  
'THEFT FROM PERSON - ATTEMPT':'THEFT','DRUNK ROLL - ATTEMPT':'DRUNK ROLL','SHOPLIFTING - ATTEMPT':'SHOPLIFTING',                                                   
'DISHONEST EMPLOYEE - GRAND THEFT':'THEFT','CREDIT CARDS, FRAUD USE ($950 & UNDER':'FRAUD','DOCUMENT WORTHLESS ($200.01 & OVER)':'DOCUMENT WORTHLESS',                                                   
'CHILD STEALING':'CHILD','THEFT FROM MOTOR VEHICLE - ATTEMPT':'THEFT', 'BATTERY ON A FIREFIGHTER':'BATTERY','THEFT, COIN MACHINE - PETTY ($950 & UNDER)':'THEFT',                                                  
'GRAND THEFT / INSURANCE FRAUD':'THEFT','BUNCO, ATTEMPT':'BUNCO','THEFT, COIN MACHINE - GRAND ($950.01 & OVER)':'THEFT',                                                   
'PURSE SNATCHING - ATTEMPT':'PURSE SNATCHING','SHOTS FIRED AT MOVING VEHICLE, TRAIN OR AIRCRAFT':'SHOTS FIRED','DISCHARGE FIREARMS/SHOTS FIRED':'SHOTS FIRED',                                                  
'CHILD ABANDONMENT':'CHILD','THEFT, COIN MACHINE - ATTEMPT':'THEFT','DISHONEST EMPLOYEE - PETTY THEFT':'THEFT','PETTY THEFT - AUTO REPAIR':'THEFT',                                                  
'TILL TAP - ATTEMPT':'TILL TAP','TILL TAP - PETTY ($950 & UNDER)':'TILL TAP','TILL TAP - GRAND THEFT ($950.01 & OVER)':'TILL TAP',                                                   
'DOCUMENT WORTHLESS ($200 & UNDER)':'DOCUMENT WORTHLESS','PICKPOCKET, ATTEMPT':'PICKPOCKET','GRAND THEFT / AUTO REPAIR':'THEFT',                                                   
'HUMAN TRAFFICKING - COMMERCIAL SEX ACTS':'HUMAN TRAFFICKING','LEWD/LASCIVIOUS ACTS WITH CHILD':'CHILD','BIKE - ATTEMPTED STOLEN':'BIKE',                                                   
'HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE':'HUMAN TRAFFICKING','CHILD PORNOGRAPHY':'CHILD','DISHONEST EMPLOYEE ATTEMPTED THEFT':'THEFT',                                                   
'FIREARMS RESTRAINING ORDER (FIREARMS RO)':'FIREARMS','VEHICLE - MOTORIZED SCOOTERS, BICYCLES, AND WHEELCHAIRS':'VEHICLE',                                                   
'FIREARMS TEMPORARY RESTRAINING ORDER (TEMP FIREARMS RO)':'FIREARMS'                                                   
})

In [6]:
#Change the dates collumn in LA data
DATE = []
for row in lapd['DATE OCC']:
    x = row.split(' ',1)
    DATE.append(x[0])
lapd.drop(columns ='DATE OCC')
DATE = pd.Series(DATE)
lapd['DATE OCC'] = DATE

In [7]:
#Used Excel to cut the years down to 2014-2017 for LA, and then narrowed that down to the most dangerous month (10/2017)
lapd = pd.read_csv("../crime_data/Cleaned_La_Crime_Data.csv")
LA_10_2017 = pd.read_csv("../crime_data/october_2017_LA_data.csv")

In [12]:
#Filter to show just violent crimes

arrests_df = og_file.loc[
    (og_file["PD_DESC"] == "AGGRAVATED HARASSMENT 1") |
    (og_file["PD_DESC"] == "AGGRAVATED HARASSMENT 2") |
    (og_file["PD_DESC"] == "ASSAULT 2,1,PEACE OFFICER") |
    (og_file["PD_DESC"] == "ASSAULT 2,1,UNCLASSIFIED") |
    (og_file["PD_DESC"] == "ASSAULT 3") |
    (og_file["PD_DESC"] == "COURSE OF SEXUAL CONDUCT AGAINST A CHILD") |
    (og_file["PD_DESC"] == "HARASSMENT,SUBD 1,CIVILIAN") |
    (og_file["PD_DESC"] == "HOMICIDE, NEGLIGENT, VEHICLE,") |
    (og_file["PD_DESC"] == "HOMICIDE, NEGLIGENT, VEHICLE, INTOX DRIVER") |
    (og_file["PD_DESC"] == "MANSLAUGHTER,UNCLASSIFIED - NON NEGLIGENT") |
    (og_file["PD_DESC"] == "MURDER,UNCLASSIFIED") |
    (og_file["PD_DESC"] == "RAPE 1") |
    (og_file["PD_DESC"] == "RAPE 2") |
    (og_file["PD_DESC"] == "RAPE 3") |
    (og_file["PD_DESC"] == "SEXUAL ABUSE 1") |
    (og_file["PD_DESC"] == "STRANGULATION 1ST")
]

#Export to csv

arrests_df.to_csv("../nyc_violent_crimes.csv")

arrests_df.head()

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
3,173113423,12/31/2017,101,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,M,18,0,25-44,M,WHITE,987831,217446,40.763523,-73.987074
4,173113421,12/31/2017,101,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,M,18,0,45-64,M,BLACK,987073,216078,40.759768,-73.989811
6,173129349,12/31/2017,101,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,M,9,0,18-24,F,WHITE HISPANIC,990592,202391,40.722199,-73.977122
8,173128437,12/31/2017,101,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,M,23,2,25-44,M,BLACK HISPANIC,998165,226184,40.787496,-73.949752
10,173128430,12/31/2017,101,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200002,M,K,83,0,25-44,M,WHITE HISPANIC,1003807,195345,40.702840,-73.929467
